# 7장 케라스 모델 고급 활용법

**감사말**

프랑소와 숄레의 [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) 7장에 사용된 코드에 대한 설명을 담고 있으며 텐서플로우 2.6 버전 이상에서 작성되었습니다. 소스코드를 공개한 저자에게 감사드립니다.

**구글 코랩 설정**

'런타임 -> 런타임 유형 변경' 메뉴에서 GPU를 지정한다.
TensorFlow 버전을 확인하려면 아래 명령문을 실행한다.

In [1]:
import tensorflow as tf
tf.__version__

'2.13.1'

TensorFlow가 GPU를 사용하는지 여부를 확인하려면 아래 명령문을 실행한다.
아래와 같은 결과가 나오면 GPU가 제대로 지원됨을 의미한다.

```
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
```

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

**참고**

아래 코드에 대한 자세한 설명은
[머신러닝 모델 고급 활용법](https://codingalzi.github.io/dlp2/working_with_keras.html)를 
참고한다.

## 7.1 케라스 활용

## 7.2 케라스 모델 구성법

### 7.2.1. 모델 구성법 1: `Sequential` 모델

**`Sequential` 클래스**

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

층의 추가는 `add` 메서드를 이용할 수도 있다.
아래 코드는 앞서 정의한 모델과 동일한 모델을 구성한다.

In [4]:
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

**모델의 가중치와 `build()` 메서드**

입력값 대신 `build()` 메서드를 특성 수 정보를 이용하여 직접 호출하면
가중치 텐서가 무작위로 초기화된 형식으로 생성된다.
즉, **모델 빌드**가 완성된다.

- `input_shape` 키워드 인자: `(None, 특성수)`
- `None`은 임의의 크기의 배치도 다룰 수 있다는 것을 의미함.

In [5]:
model.build(input_shape=(None, 3))

**층별 가중치 텐서**

모델 빌드가 완성되면 `weights` 속성에 생성된 모델 훈련에 필요한 모든 가중치와 편향이 저장된다.
위 모델에 대해서 층별로 가중치와 편향 텐서 하나씩 총 4 개의 텐서가 생성된다.

In [6]:
len(model.weights)

4

In [7]:
model.weights

[<tf.Variable 'dense_2/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[ 0.25079608, -0.02053735,  0.04458058, -0.16679166,  0.10456336,
          0.2889604 , -0.10949062, -0.04367691,  0.01961142, -0.18474612,
         -0.27756682, -0.06971286, -0.16615126,  0.0642336 , -0.2394494 ,
         -0.09103076,  0.21065146,  0.0573442 ,  0.06813774,  0.2763595 ,
          0.19246206,  0.20148784, -0.19099021, -0.18175165,  0.18369201,
          0.24098158,  0.10313791,  0.1955778 ,  0.1441271 ,  0.26645273,
         -0.2539499 , -0.28577235, -0.06036349,  0.0489656 , -0.29800895,
         -0.11436272,  0.16543606,  0.11120889,  0.13089806, -0.04786992,
          0.05420664,  0.10066786,  0.0410319 , -0.05855761, -0.22293767,
         -0.10142365,  0.2202195 ,  0.06329516,  0.1843049 ,  0.11907443,
         -0.04896745, -0.01138321, -0.00735086,  0.22307903,  0.21142977,
          0.2768681 , -0.1852521 , -0.15613721,  0.15851188, -0.01811898,
          0.14916301,  0.2342183 , -0.19237

- 1층의 가중치와 편향 텐서

In [8]:
model.weights[0].shape

TensorShape([3, 64])

In [9]:
model.weights[1].shape

TensorShape([64])

- 2층의 가중치와 편향 텐서

In [10]:
model.weights[2].shape

TensorShape([64, 10])

In [11]:
model.weights[3].shape

TensorShape([10])

**`summary()` 메서드**

완성된 모델의 요악한 내용은 확인할 수 있다.

- 모델과 층의 이름
- 층별 파라미터 수
- 파라미터 수

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                256       
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**`Input()` 함수**

모델 구성 중간에 구성 과정을 확인하려면 `Input()`함수를 이용하여
**케라스텐서**(`KerasTensor`) 객체를
가장 먼저 모델에 추가한다.
그러면 층을 추가할 때마다 `summary()`를 실행할 수 있다.

In [13]:
model = keras.Sequential()
model.add(keras.Input(shape=(3,)))
model.add(layers.Dense(64, activation="relu"))

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
Total params: 256 (1.00 KB)
Trainable params: 256 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.add(layers.Dense(10, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 7.2.2. 모델 구성법 2: 함수형 API

**기본 활용법**

앞서 살펴 본 `Sequential` 모델을 함수형 API를 이용하여 구성하면 다음과 같다.

In [16]:
inputs = keras.Input(shape=(3,), name="my_input")          # 입력층
features = layers.Dense(64, activation="relu")(inputs)     # 은닉층
outputs = layers.Dense(10, activation="softmax")(features) # 출력층

model = keras.Model(inputs=inputs, outputs=outputs)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_6 (Dense)             (None, 64)                256       
                                                                 
 dense_7 (Dense)             (None, 10)                650       
                                                                 
Total params: 906 (3.54 KB)
Trainable params: 906 (3.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**다중 입력, 다중 출력 모델**

예제: 고객 요구사항 접수 모델

고객의 요구사항의 처리할 때 필요한 우선순위와 담당부서를 지정하는 시스템을 구현하려 한다. 시스템에 사용될 딥러닝 모델은 세 개의 입력과 두 개의 출력을 사용한다.

In [18]:
vocabulary_size = 10000    # 사용빈도 1만등 인내 단어 사용
num_tags = 100             # 태그 수
num_departments = 4        # 부서 수

# 입력층: 세 개
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

# 은닉층
features = layers.Concatenate()([title, text_body, tags]) # shape=(None, 10000+10000+100)
features = layers.Dense(64, activation="relu")(features)

# 출력층: 두 개
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
    num_departments, activation="softmax", name="department")(features)

# 모델 빌드: 입력값으로 구성된 입력값 리스트와 출력값으로 구성된 출력값 리스트 사용
model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

은닉층으로 사용된 `Concatenate()` 층은 여러 개의 텐서를 좌우로 이어붙이는 기능을 수행한다.
따라서 여기서는 고객의 요구사항을 담고 있는
길이가 각각 1만, 1만, 100인 세 개의 벡터를 이어붙여서 길이가 총 2만1백인 하나의 벡터로 변환한다.

모델 훈련을 위해 적절한 개수의 입력 텐서와 타깃 텐서를 지정해야 한다.
여기서는 훈련 과정을 설명하기 위해 
적절한 모양의 입력 텐서 3개와 타깃 텐서 2개를 무작위로 생성해서 사용한다.

In [19]:
import numpy as np

# 샘플 수
num_samples = 1280

# 입력 텐서 3 개 무작위 생성
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))    # 멀티-핫-인코딩

# 타깃 텐서 2 개 무작위 생성
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))  # 멀티-핫-인코딩

모델 컴파일 과정에서 지정된 타깃 수만큼 손실함수와 측정 기준을 지정해야 한다.

In [20]:
model.compile(optimizer="adam",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

모델 훈련은 `fit()` 함수에 세 개의 훈련 텐서로 이루어진 리스트와 
두 개의 타깃 텐서로 이루어진 리스트를 지정한 후에 실행한다. 

In [21]:
model.fit([title_data, text_body_data, tags_data],
          [priority_data, department_data],
          epochs=10)

Epoch 1/10
40/40 [==============================] - 6s 9ms/step - loss: 13.2260 - priority_loss: 0.3347 - department_loss: 12.8913 - priority_mean_absolute_error: 0.5004 - department_accuracy: 0.2055
Epoch 2/10
40/40 [==============================] - 0s 4ms/step - loss: 4.8690 - priority_loss: 0.3455 - department_loss: 4.5235 - priority_mean_absolute_error: 0.5130 - department_accuracy: 0.3414
Epoch 3/10
40/40 [==============================] - 0s 4ms/step - loss: 10.3178 - priority_loss: 0.3455 - department_loss: 9.9723 - priority_mean_absolute_error: 0.5130 - department_accuracy: 0.3344
Epoch 4/10
40/40 [==============================] - 0s 4ms/step - loss: 18.2820 - priority_loss: 0.3455 - department_loss: 17.9365 - priority_mean_absolute_error: 0.5130 - department_accuracy: 0.2547
Epoch 5/10
40/40 [==============================] - 0s 4ms/step - loss: 19.3723 - priority_loss: 0.3455 - department_loss: 19.0268 - priority_mean_absolute_error: 0.5130 - department_accuracy: 0.2805
Epo

모델 평가도 훈련과 동일한 방식의 인자가 사용된다.

In [22]:
model.evaluate([title_data, text_body_data, tags_data],
               [priority_data, department_data])

40/40 [==============================] - 0s 9ms/step - loss: 63.2337 - priority_loss: 0.3455 - department_loss: 62.8882 - priority_mean_absolute_error: 0.5130 - department_accuracy: 0.5594


[63.23370361328125,
 0.3455060124397278,
 62.88820266723633,
 0.5129579305648804,
 0.559374988079071]

예측값은 두 개의 어레이로 구성된 리스트이다.

In [23]:
priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

40/40 [==============================] - 0s 6ms/step


In [24]:
priority_preds

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [25]:
department_preds

array([[9.9997675e-01, 0.0000000e+00, 2.0660605e-06, 2.1195068e-05],
       [9.4204134e-01, 0.0000000e+00, 7.1774691e-04, 5.7240859e-02],
       [8.3637291e-01, 0.0000000e+00, 8.5415365e-04, 1.6277289e-01],
       ...,
       [5.2222282e-01, 0.0000000e+00, 4.2686789e-04, 4.7735029e-01],
       [9.6426433e-01, 0.0000000e+00, 3.6303685e-03, 3.2105319e-02],
       [9.9827170e-01, 0.0000000e+00, 2.7596447e-05, 1.7006756e-03]],
      dtype=float32)

**사전 객체 활용**

입력층과 출력층의 이름을 이용하여 사전 형식으로 입력값과 출력값을 지정할 수 있다.

In [26]:
model.compile(optimizer="adam",
              loss={"priority": "mean_squared_error", "department": "categorical_crossentropy"},
              metrics={"priority": ["mean_absolute_error"], "department": ["accuracy"]})

model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
          {"priority": priority_data, "department": department_data},
          epochs=1)

model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data},
               {"priority": priority_data, "department": department_data})

priority_preds, department_preds = model.predict(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})

40/40 [==============================] - 0s 2ms/step


**층 연결 구조**

`plot_model()`을 이용하여 층 연결 구조를 그래프로 나타낼 수 있다.

```python
>>> keras.utils.plot_model(model, "ticket_classifier.png")
```

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/ticket_classifier.png" style="width:400px;"></div>

입력 텐서와 출력 텐서의 모양을 함께 표기할 수도 있다.

```python
>>> keras.utils.plot_model(model, "ticket_classifier_with_shape_info.png", show_shapes=True)
```

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/ticket_classifier_with_shapes.png" style="width:900px;"></div>

**모델 재활용**

훈련된 모델의 특성을 이용하여 새로운 모델을 빌드할 수 있다.
먼저 모델의 `layers` 속성을 이용하여 사용된 층에 대한 정보를 확인한다. 
`layers` 속성은 사용된 층들의 객체로 이루어진 리스트를 가리킨다.

In [27]:
model.layers

예를 들어, 3번 인덱스에 해당하는 층의 입력값과 출력값에 대한 정보는 아래처럼 확인할 수 있다.

In [28]:
model.layers[3].input

[<KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'title')>,
 <KerasTensor: shape=(None, 10000) dtype=float32 (created by layer 'text_body')>,
 <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'tags')>]

In [29]:
model.layers[3].output

<KerasTensor: shape=(None, 20100) dtype=float32 (created by layer 'concatenate')>

출력층을 제외한 나머지 층을 재활용해보자.
출력층은 5번과 6번 인덱스에 위치하기에 4번 인덱스가
가리키는 (은닉)층의 출력 정보를 따로 떼어낸다.

In [30]:
features = model.layers[4].output

이제 출력층에 문제해결의 어려움 정도를 "quick", "medium", "difficult"로
구분하는 어려움(difficulty) 정도를 판별하는 층을 추가해보자.
먼저, `difficulty` 층을 준비한다.

In [31]:
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)

준비된 `'difficulty'` 층을 출력층으로 추가하여 
`priority`, `department`, `difficulty`
세 개의 출력값을 생성하는 새로운 모델을 구성한다.

In [32]:
new_model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department, difficulty])

새로 생성된 모델은 기존에 훈련된 모델의 가중치,
즉, 은닉층에 사용된 가중치는 그대로 사용되며,
모델 구성 그래프는 다음과 같다.

```python
>>> keras.utils.plot_model(new_model, "updated_ticket_classifier.png", show_shapes=True)
```

<div align="center"><img src="https://drek4537l1klr.cloudfront.net/chollet2/v-7/Figures/updated_ticket_classifier.png" style="width:900px;"></div>

요약 결과는 다음과 같다.

In [33]:
new_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 title (InputLayer)          [(None, 10000)]              0         []                            
                                                                                                  
 text_body (InputLayer)      [(None, 10000)]              0         []                            
                                                                                                  
 tags (InputLayer)           [(None, 100)]                0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 20100)                0         ['title[0][0]',               
                                                                     'text_body[0][0]',     

### 모델 구성법 3: 서브클래싱

앞서 함수형 API로 구성한 티켓 모델을 서브클래싱을 기법을 이용하여 구현하면 다음과 같다.

In [34]:
class CustomerTicketModel(keras.Model):

    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(
            num_departments, activation="softmax")

    def call(self, inputs):               # inputs: 사전 객체 입력값. 모양은 미정.
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])    # 은닉층
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)                 # 출력층
        department = self.department_classifier(features)
        return priority, department                               # outputs

모델 구성은 해당 모델의 객체를 생성하면 된다.
다만 `Layer`의 경우처럼 가중치는 실제 데이터와 함께 호출되지 전까지 생성되지 않는다.

In [35]:
model = CustomerTicketModel(num_departments=4)

model.weights

[]

컴파일, 훈련, 평가, 예측은 이전과 완전히 동일한 방식으로 실행된다.

In [36]:
model.compile(optimizer="adam",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])
model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
          [priority_data, department_data],
          epochs=1)
model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data},
               [priority_data, department_data])
priority_preds, department_preds = model.predict(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})

40/40 [==============================] - 0s 2ms/step


### 7.2.4. 혼합 모델 구성법

소개된 세 가지 방식을 임의로 혼합하여 활용할 수 있다. 

**예제: 서브클래싱 모델을 함수형 모델에 활용하기** (강추!!!)

In [37]:
class Classifier(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        if num_classes == 2:
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"
        self.dense = layers.Dense(num_units, activation=activation)

    def call(self, inputs):
        return self.dense(inputs)

inputs = keras.Input(shape=(3,))
features = layers.Dense(64, activation="relu")(inputs)
outputs = Classifier(num_classes=10)(features)
model = keras.Model(inputs=inputs, outputs=outputs)

**예제: 함수형 모델을 서브클래싱 모델에 활용하기**

In [38]:
inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation="sigmoid")(inputs)
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)

class MyModel(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation="relu")
        self.classifier = binary_classifier

    def call(self, inputs):
        features = self.dense(inputs)
        return self.classifier(features)

model = MyModel()

## 7.3 모델 훈련/평가 방식 조정

케라스 모델의 구성, 훈련, 평가, 예측은 정해진 방식으로 차례대로 이루어진다.
아래 코드는 MNIST 데이터셋을 이용한 모델 훈련 전반 과정을 보여준다.

In [39]:
from tensorflow.keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 [==============================] - 6s 3ms/step - loss: 0.2961 - accuracy: 0.9121 - val_loss: 0.1649 - val_accuracy: 0.9548
Epoch 2/3
1563/1563 [==============================] - 4s 3ms/step - loss: 0.1605 - accuracy: 0.9535 - val_loss: 0.1170 - val_accuracy: 0.9677
Epoch 3/3
313/313 [==============================] - 0s 1ms/step


### 7.3.1. 사용자 정의 평가지표 활용

**`Metric` 클래스 상속**

아래 세 개의 메서드를 재정의(overriding)해야 한다.

- `update_state()`
- `result()`
- `reset_state()`

아래 코드는 평균제곱근오차(RMSE)를 평가지표로 사용하는 클래스를 
이용하는 모델 훈련을 소개한다.

In [40]:
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):

    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [41]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RootMeanSquaredError()])
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.2992 - accuracy: 0.9114 - rmse: 7.1725 - val_loss: 0.1467 - val_accuracy: 0.9568 - val_rmse: 7.3521
Epoch 2/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.1607 - accuracy: 0.9530 - rmse: 7.3501 - val_loss: 0.1183 - val_accuracy: 0.9656 - val_rmse: 7.3993
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.0968 - accuracy: 0.9720 - rmse: 7.4287


### 7.3.2. 콜백 활용

**콜백**(callback)은 모델 훈련 도중에 부가적으로 호출되는 객체이며
학습 과정을 모니터링 하면서 일부 제어기능을 수행하는 다양한 메서드를 제공한다.
콜백이 활용되는 주요 기능은 다음과 같다.

- 모델 체크포인팅: 훈련 중 모델 상태 수시로 저장
- 훈련 조기 중단: 검증셋 손실이 더 이상 개선되지 않는 경우 훈련 중단
- 하이퍼 파라미터 조정: 학습률의 동적 변경
- 훈련 기록 작성: 훈련셋 및 검증셋의 손실값, 평가지표 등 기록 및 시각화

```python
keras.callbacks.ModelCheckpoint
keras.callbacks.EarlyStopping
keras.callbacks.LearningRateScheduler
keras.callbacks.ReduceLROnPlateau
keras.callbacks.CSVLogger
```

여기서는 `EarlyStopping`과 `ModelCheckpoint` 두 콜백의 기능을 살펴본다.

**`fit()` 메서드에서 `callbacks` 인자 사용하기**

아래 코드에 사용된 옵션은 다음과 같다.

- `EarlyStopping`: 검증셋에 대한 정확도가 2 에포크(epoch) 연속 개선되지 않을 때 훈련 종료
- `ModelCheckpoint`: 매 에포크마다 훈련된 모델 저장. 
    `save_best_only=True`가 설정된 경우 검증셋에 대한 손실값이 가장 낮은 모델만 저장.

In [42]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]

In [45]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list,
          validation_data=(val_images, val_labels))

Epoch 1/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2936 - accuracy: 0.9129 - val_loss: 0.1345 - val_accuracy: 0.9600
Epoch 2/10
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1588 - accuracy: 0.9539 - val_loss: 0.1142 - val_accuracy: 0.9675
Epoch 3/10
1549/1563 [============================>.] - ETA: 0s - loss: 0.1287 - accuracy: 0.9640

ValueError: The following argument(s) are not supported with the native Keras format: ['options']

조기종료 후 훈련과정에서 저장된 최고 성능의 모델을 불러오면 다음과 같다.

In [ ]:
model = keras.models.load_model("checkpoint_path.keras")

### 7.3.3. 사용자 정의 콜백 활용

**`Callback` 클래스 상속**

매 에포크와 매 배치 훈련 단계의 시작과 종료 지점에서
수행해야 할 기능을 정의해야 하며 아래 메서드를 재정의하는 방식으로 이루어진다.

```python
on_epoch_begin(epoch, logs)
on_epoch_end(epoch, logs)
on_batch_begin(batch, logs)
on_batch_end(batch, logs)
on_train_begin(logs)
on_train_end(logs)
```

각 메서드에 사용되는 인자는 훈련 과정 중에 자동으로 생성된 객체로부터 값을 받아온다.

- `logs` 인자: 이전 배치와 에포크의 훈련셋과 검증셋에 대한 손실값, 평가지표 등을 포함한 사전 객체.
- `batch`, `epoch`: 배치와 에포크 정보

다음 `LossHistory` 콜백 클래스는 배치 훈련이 끝날 때마다 손실값을 저장하고
에포크가 끝날 때마다 배치별 손실값을 그래프로 저장하여 훈련이 종료된 후 시각화하여 보여주도록 한다.

In [ ]:
from matplotlib import pyplot as plt

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.per_batch_losses = []

    def on_batch_end(self, batch, logs):
        self.per_batch_losses.append(logs.get("loss"))

    def on_epoch_end(self, epoch, logs):
        plt.clf()
        plt.plot(range(len(self.per_batch_losses)), self.per_batch_losses,
                 label="Training loss for each batch")
        plt.xlabel(f"Batch (epoch {epoch})")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(f"plot_at_epoch_{epoch}")
        self.per_batch_losses = []

In [ ]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=[LossHistory()],
          validation_data=(val_images, val_labels))

### 7.3.4. 텐서보드(TensorBoard) 활용

**텐서보드**(TensorBoard)는 모델 훈련과정을 모니터링하는 최고의 어플이며
텐서플로우와 함께 기본적으로 설치된다.

**주의사항**: 텐서보드 데이터의 저장경로를 

```python
/full_path_to_your_log_dir
```

대신에 

```python
./tensorboard_log_dir
```

등을 사용해야 리눅스, 맥 운영체제에서 오류가 발생하지 않는다.

In [ ]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

tensorboard = keras.callbacks.TensorBoard(
    log_dir="./tensorboard_log_dir",
)

model.fit(train_images, train_labels,
          epochs=10,
          validation_data=(val_images, val_labels),
          callbacks=[tensorboard])

텐서보드를 주피터 노트북에서 아래처럼 실행할 수 있다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./tensorboard_log_dir

텐서보드를 독립적으로 실행하여 훈련과정을 실시간으로 모니터링 하려면
아래 명령어를 터미널 창에서 실행하고 반환된 주소로 접속하면 된다.

```python
tensorboard --logdir ./full_path_to_your_log_dir
```

## 7.4 훈련/평가 알고리즘 직접 구현하기

모델 컴파일 이후 `fit()` 메서드를 호출하면 모델의 훈련이 진행된다.
그런데 모델의 훈련 방식을 사용자가 직접 조정할 수 있는 방식으로 진행하고자 하면
아래 과정을 자신만의 알고리즘으로 직접 구현하면 된다.

- 순전파<font size='2'>forward pass</font>
- 손실함수의 그레이디언트 계산
- 역전파<font size='2'>backward pass</font>

### Low-level usage of metrics

In [ ]:
metric = keras.metrics.SparseCategoricalAccuracy()
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f"result: {current_result:.2f}")

In [ ]:
values = [0, 1, 2, 3, 4]
mean_tracker = keras.metrics.Mean()
for value in values:
    mean_tracker.update_state(value)
print(f"Mean of values: {mean_tracker.result():.2f}")

### A complete training and evaluation loop

**Writing a step-by-step training loop: the training step function**

In [ ]:
model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = keras.metrics.Mean()

def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["loss"] = loss_tracking_metric.result()
    return logs

**Writing a step-by-step training loop: resetting the metrics**

In [ ]:
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

**Writing a step-by-step training loop: the loop itself**

In [ ]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.batch(32)
epochs = 3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"Results at the end of epoch {epoch}")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

**Writing a step-by-step evaluation loop**

In [ ]:
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("Evaluation results:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

### Make it fast with `tf.function`

**Adding a `tf.function` decorator to our evaluation step function**

In [ ]:
@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print("Evaluation results:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

### Leveraging `fit()` with a custom training loop

**Implementing a custom training step to use with `fit()`**

In [ ]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
loss_tracker = keras.metrics.Mean(name="loss")

class CustomModel(keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        loss_tracker.update_state(loss)
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        return [loss_tracker]

In [ ]:
inputs = keras.Input(shape=(28 * 28,))
features = layers.Dense(512, activation="relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.compiled_loss(targets, predictions)
        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        self.compiled_metrics.update_state(targets, predictions)
        return {m.name: m.result() for m in self.metrics}

In [ ]:
inputs = keras.Input(shape=(28 * 28,))
features = layers.Dense(512, activation="relu")(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=3)